In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[7],5
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.5e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[1, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000200                    1 -0.000235
0.000750        2 -0.000174                    2 -0.000162
0.001052        3 -0.000106                    3 -0.000202
0.001476        4 -0.000011                    4 -0.000146
0.002070        5  0.000115                    5 -0.000070
0.002904        6  0.000280                    6  0.000027
0.004074        7  0.000491                    7  0.000155
0.005714        8  0.000758                    8  0.000326
0.008015        9  0.001092                    9  0.000548
0.011243       10  0.001506                   10  0.000823
0.015771       11  0.002014                   11  0.001160
0.022122       12  0.002634                   12  0.001573
0.031031       13  0.003381                   13  0.002069
0.043528       14  0.004169                   14  0.002592
0.061057       15  0.004861                   15  0.003061
0.085645       16  0.005526                   16  0.003517
0.120136       17  0.006249                   17  0.004011
0.168516       18  0.007043                   18  0.004538
0.236378       19  0.007909                   19  0.005095
0.331549       20  0.008852                   20  0.005685
0.465100       21  0.009765                   21  0.006303
0.652400       22  0.008928                   22  0.005890
0.915100       23  0.006238                   23  0.004165
1.283650       24  0.003852                   24  0.002568
1.800600       25  0.002231                   25  0.001457
2.525700       26  0.001160                   26  0.000715
3.542800       27  0.000479                   27  0.000235
4.969550       28  0.000107                   28 -0.000031
6.970850       29 -0.000072                   29 -0.000163
9.778100       30 -0.000174                   30 -0.000241
13.715850      31 -0.000246                   31 -0.000297
19.239350      32 -0.000295                   32 -0.000335
26.987250      33 -0.000317                   33 -0.000352
37.855300      34 -0.000322                   34 -0.000352
53.100050      35 -0.000327                   35 -0.000371
73.887500      36 -0.000337                   36 -0.000393
97.662500      37 -0.000352                   37 -0.000404
121.437500     38 -0.000377                   38 -0.000411
145.212500     39 -0.000406                   39 -0.000414
168.987500     40 -0.000431                   40 -0.000416
192.762500     41 -0.000454                   41 -0.000420
216.537500     42 -0.000475                   42 -0.000433
240.312500     43 -0.000494                   43 -0.000449
264.087500     44 -0.000512                   44 -0.000462
287.862500     45 -0.000526                   45 -0.000470
311.637500     46 -0.000517                   46 -0.000457
335.412500     47 -0.000462                   47 -0.000399
359.187500     48 -0.000362                   48 -0.000302
382.962500     49 -0.000226                   49 -0.000175
406.737500     50 -0.000054                   50 -0.000012
430.512500     51  0.000156                   51  0.000184
454.287500     52  0.000403                   52  0.000414
478.062500     53  0.000687                   53  0.000682
501.837500     54  0.001008                   54  0.000987
525.612500     55  0.001362                   55  0.001310
549.387500     56  0.001745                   56  0.001663
573.162500     57  0.002152                   57  0.002044
596.937500     58  0.002576                   58  0.002448
620.712500     59  0.003008                   59  0.002867
644.487500     60  0.003440                   60  0.003278
668.262500     61  0.003864                   61  0.003673
692.037500     62  0.004278                   62  0.004064
715.812500     63  0.004678                   63  0.004449
739.587500     64  0.005065                   64  0.004827
763.362500     65  0.005438                   65  0.005198
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -14.967317  0.000000e+00 -14.967317 -14.974843   
0.000624    2     -14.967317  2.991724e-08 -14.967317 -14.974843   
0.000876    3     -14.967317  4.318437e-08 -14.967317 -14.974843   
0.001229    4     -14.967317  6.574801e-08 -14.967317 -14.974843   
0.001723    5     -14.967317  1.041414e-07 -14.967317 -14.974843   
0.002417    6     -14.967317  1.692970e-07 -14.967317 -14.974843   
0.003391    7     -14.967317  2.796168e-07 -14.967317 -14.974843   
0.004757    8     -14.967317  4.659454e-07 -14.967317 -14.974843   
0.006672    9     -14.967317  7.798133e-07 -14.967316 -14.974843   
0.009359    10    -14.967317  1.307312e-06 -14.967316 -14.974843   
0.013128    11    -14.967318  2.191840e-06 -14.967315 -14.974844   
0.018415    12    -14.967318  3.671762e-06 -14.967314 -14.974844   
0.025830    13    -14.967318  6.142377e-06 -14.967312 -14.974844   
0.036232    14    -14.967319  1.025478e-05 -14.967308 -14.974845   
0.050823    15    -14.967318  1.695316e-05 -14.967301 -14.974845   
0.071291    16    -14.967317  2.746729e-05 -14.967289 -14.974845   
0.100000    17    -14.967314  4.368956e-05 -14.967270 -14.974845   
0.140271    18    -14.967309  6.863946e-05 -14.967241 -14.974844   
0.196760    19    -14.967300  1.069373e-04 -14.967193 -14.974841   
0.275997    20    -14.967284  1.655552e-04 -14.967119 -14.974835   
0.387100    21    -14.967257  2.549457e-04 -14.967003 -14.974824   
0.543100    22    -14.967212  3.896073e-04 -14.966822 -14.974804   
0.761700    23    -14.967154  5.626884e-04 -14.966591 -14.974781   
1.068500    24    -14.967106  7.419751e-04 -14.966365 -14.974770   
1.498800    25    -14.967080  9.130413e-04 -14.966168 -14.974781   
2.102400    26    -14.967082  1.073801e-03 -14.966009 -14.974817   
2.949000    27    -14.967115  1.222827e-03 -14.965892 -14.974883   
4.136600    28    -14.967184  1.359788e-03 -14.965824 -14.974985   
5.802500    29    -14.967296  1.492641e-03 -14.965803 -14.975129   
8.139200    30    -14.967461  1.637298e-03 -14.965824 -14.975327   
11.417000   31    -14.967697  1.806295e-03 -14.965891 -14.975599   
16.014700   32    -14.968034  2.009445e-03 -14.966024 -14.975971   
22.464000   33    -14.968513  2.263184e-03 -14.966249 -14.976480   
31.510500   34    -14.969210  2.620353e-03 -14.966590 -14.977194   
44.200100   35    -14.970257  3.183460e-03 -14.967074 -14.978216   
62.000000   36    -14.971844  4.080500e-03 -14.967763 -14.979784   
85.775000   37    -14.974132  5.421049e-03 -14.968711 -14.982100   
109.550000  38    -14.976573  6.871210e-03 -14.969702 -14.984558   
133.325000  39    -14.979125  8.361958e-03 -14.970763 -14.987079   
157.100000  40    -14.981760  9.854111e-03 -14.971905 -14.989590   
180.875000  41    -14.984465  1.134609e-02 -14.973119 -14.992093   
204.650000  42    -14.987236  1.283788e-02 -14.974399 -14.994603   
228.425000  43    -14.990065  1.432948e-02 -14.975736 -14.997178   
252.200000  44    -14.992949  1.582082e-02 -14.977127 -14.999835   
275.975000  45    -14.995882  1.731253e-02 -14.978570 -15.002556   
299.750000  46    -14.998872  1.882206e-02 -14.980050 -15.005333   
323.525000  47    -15.001981  2.047547e-02 -14.981506 -15.008218   
347.300000  48    -15.005285  2.247730e-02 -14.982808 -15.011233   
371.075000  49    -15.008791  2.496198e-02 -14.983828 -15.014403   
394.850000  50    -15.012461  2.799557e-02 -14.984466 -15.017720   
418.625000  51    -15.016254  3.163758e-02 -14.984617 -15.021130   
442.400000  52    -15.020125  3.594687e-02 -14.984178 -15.024567   
466.175000  53    -15.024023  4.098056e-02 -14.983043 -15.028011   
489.950000  54    -15.027901  4.679276e-02 -14.981108 -15.031433   
513.725000  55    -15.031703  5.343255e-02 -14.978271 -15.034796   
537.500000  56    -15.035378  6.094201e-02 -14.974436 -15.038004   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')